In [1]:
from scqbf.scqbf_instance import *
from scqbf.scqbf_solution import *
from scqbf.scqbf_evaluator import *
from scqbf.scqbf_ts import *

import pandas as pd
import numpy as np
import pickle

In [2]:
instance_paths = [(f"instances/gen{i}/instance{j}.txt", i, j) for i in range(1, 4) for j in range(1, 6)]
instance_paths

[('instances/gen1/instance1.txt', 1, 1),
 ('instances/gen1/instance2.txt', 1, 2),
 ('instances/gen1/instance3.txt', 1, 3),
 ('instances/gen1/instance4.txt', 1, 4),
 ('instances/gen1/instance5.txt', 1, 5),
 ('instances/gen2/instance1.txt', 2, 1),
 ('instances/gen2/instance2.txt', 2, 2),
 ('instances/gen2/instance3.txt', 2, 3),
 ('instances/gen2/instance4.txt', 2, 4),
 ('instances/gen2/instance5.txt', 2, 5),
 ('instances/gen3/instance1.txt', 3, 1),
 ('instances/gen3/instance2.txt', 3, 2),
 ('instances/gen3/instance3.txt', 3, 3),
 ('instances/gen3/instance4.txt', 3, 4),
 ('instances/gen3/instance5.txt', 3, 5)]

In [5]:
def run_experiment(config: TSConfig) -> pd.DataFrame:    
    results = []
    exp_num = 1

    for instance_path, gen, inst in instance_paths:
        instance = read_max_sc_qbf_instance(instance_path)
        print(f"{exp_num}: {inst}th instance of generation strategy {gen}. Path: {instance_path}")
        exp_num += 1
        
        if config.ibr_component is not None:
            config.ibr_component._instance = instance
        
        time_limit = 600*3
        ts = ScQbfTS(instance, config, time_limit_secs=time_limit, patience=1000)
        best_solution = ts.solve()
        
        evaluator = ScQbfEvaluator(instance)
        results.append({
            'gen': gen,
            'inst': inst,
            'n': instance.n,
            'stop_reason': ts.stop_reason,
            'best_objective': evaluator.evaluate_objfun(best_solution),
            'coverage': evaluator.evaluate_coverage(best_solution),
            'time_taken': ts.execution_time
        })
        
        last_result = results[-1]
        print(f"\tBest objective value: {last_result['best_objective']:.4f}")
        print(f"Selected elements: {best_solution.elements}")
        print(f"\tCoverage: {last_result['coverage']:.2%}")
        print(f"\tTime taken (secs): {last_result['time_taken']:.4f} s")
        print(f"\tStop reason: {last_result['stop_reason']}")
        print()
    
    df = pd.DataFrame(results)
    return df

In [4]:
# Tenure parameters

T1 = 5
T2 = 10

# Experimento 1 - PADRÃO
Busca Tabu com método de busca first-improving, “tabu tenure” igual a T1 , estratégia
tabu padrão.

In [ ]:
config = TSConfig(tenure=T1, search_strategy='first')
results_df = run_experiment(config)

with open("pickles/results_config_1.pkl", "wb") as f:
    pickle.dump(results_df, f)

display(results_df)

1: 1th instance of generation strategy 1. Path: instances/gen1/instance1.txt
	Best objective value: 4609.8800
Selected elements: [110, 126, 147, 112, 150, 28, 48, 171, 105, 36, 87, 163, 65, 116, 160, 135, 54, 123, 159, 72, 60, 71, 67, 161, 12, 46, 148, 137, 27, 172, 76, 196, 179, 69, 38, 79, 5, 192, 90, 102, 15, 97, 189, 169, 167, 68, 23, 157, 187, 145, 92, 182, 184, 11, 22, 151, 113, 139, 152, 86, 173, 94, 115, 13, 109, 98, 195, 80, 9, 81, 78, 178, 32, 146, 50, 111, 117, 14, 51, 114, 198, 45, 83, 134, 154, 144, 47, 136, 131, 25, 61, 174, 37, 158, 73, 175, 85, 64, 127, 166, 53, 3, 33, 40, 153, 43, 8, 142, 89, 132, 41, 133, 10, 118, 129, 91, 99, 128, 1, 44, 197]
	Coverage: 100.00%
	Time taken (secs): 10.1570 s
	Stop reason: time_limit

2: 2th instance of generation strategy 1. Path: instances/gen1/instance2.txt
	Best objective value: 4622.6900
Selected elements: [144, 57, 82, 122, 46, 177, 162, 67, 156, 172, 97, 45, 179, 29, 93, 121, 23, 21, 42, 84, 185, 15, 10, 106, 58, 180, 11, 178, 1

,gen,inst,n,stop_reason,best_objective,coverage,time_taken
0,1,1,200,time_limit,4609.88,1.0,10.157021
1,1,2,200,time_limit,4622.69,1.0,10.111395
2,1,3,50,patience_exceeded,642.03,1.0,3.032495
3,1,4,25,patience_exceeded,261.06,1.0,0.478303
4,1,5,100,time_limit,1476.25,1.0,10.014869
5,2,1,50,patience_exceeded,562.74,1.0,2.967328
6,2,2,400,time_limit,14125.45,1.0,10.445429
7,2,3,25,patience_exceeded,210.16,1.0,0.487231
8,2,4,400,time_limit,13261.32,1.0,10.042966
9,2,5,100,time_limit,1515.77,1.0,10.005226


# Experimento 2 - PADRÃO+BEST
Busca Tabu PADRÃO mas com método de busca best-improving.

In [ ]:
config = TSConfig(tenure=T1, search_strategy='best')
results_df = run_experiment(config)

with open("pickles/results_config_2.pkl", "wb") as f:
    pickle.dump(results_df, f)

display(results_df)

# Experimento 3 - PADRÃO+TENURE
Busca Tabu PADRÃO mas com novo valor de “tabu tenure” igual a T2 .

In [ ]:
config = TSConfig(tenure=T2, search_strategy='first')
results_df = run_experiment(config)

with open("pickles/results_config_3.pkl", "wb") as f:
    pickle.dump(results_df, f)

display(results_df)

# Experimento 4 - PADRÃO+METHOD1
Busca Tabu PADRÃO mas com estratégia tabu alternativa 1 (Probabilistic TS)

In [ ]:
config = TSConfig(tenure=T1, search_strategy='first', probabilistic_ts=True, probabilistic_param=0.6)
results_df = run_experiment(config)

with open("pickles/results_config_3.pkl", "wb") as f:
    pickle.dump(results_df, f)

display(results_df)

# Experimento 5 - PADRÃO+METHOD2
Busca Tabu PADRÃO mas com estratégia tabu alternativa 2 (Intesification by Restart).

In [ ]:
ibr_component = RestartIntensificationComponent(restart_patience=100, max_fixed_elements=5)
config = TSConfig(tenure=T1, search_strategy='first', ibr_component=ibr_component)
results_df = run_experiment(config)

with open("pickles/results_config_3.pkl", "wb") as f:
    pickle.dump(results_df, f)

display(results_df)